# Expected Value of a Slot Machine

A slot machine with, in this example 3 slots and 4 different symbols can have 64 (4 x 4 x 4) different combinations. The number of combinations in today's slots is somewhat higher. For example 5 slots with 30 symbols each, has a total of combinations of 243,000,000.

In [1]:
symbols = [1, 2, 3, 4]
spins = [1, 2, 3]

In [2]:
def total_combinations(symbols: list, spins: list) -> int:
    """ Calculates all possible combinations
    :param symbols: list of different symbols
    :param spins: list that represents the number of slots
    """
    result = 0
    for i, spin in enumerate(spins):
        if i == 0:
            result = len(symbols)
        else:
            result = result * len(symbols)
    return result

In [3]:
print("Number of possible combinations are", total_combinations(symbols, spins))

Number of possible combinations are 64


The possible winning combinations in this example are straight forward:<br/>
1. Three same symbols win 50 coins<br/>
2. Two same symbols win of 10 coins<br/>

In [4]:
import random

def spin(symbols: list, spins: list) -> int:
    """ One slot machine spin
    :param symbols: list of different symbols
    :param spins: list that represents the number of slots
    """
    result = []
    for spin in spins:
        result.append(random.choice(symbols))
    if(len(set(result)) < 3):
        if(len(set(result)) < 2):
            return(50)
        else:
            return(10)
    else:
        return(0)

In [5]:
spin(symbols, spins)

0

The number of winning combinations are:<br/>
1. For each spinning slot with three times the same pairs gives 4 combinations.<br/>
2. The second combination requires just 2 times the same symbol on any slot 4 times (1 x 1 x 4) + (1 x 4 x 1) + (4 x 1 x 1) = 36 combinations.

In [6]:
import itertools

def winning_combinations(symbols: list, spins: list, n_same_symbols:int) -> int:
    """ Calculates the number of combinations.
    
    :param symbols: list of different symbols
    :param spins: list that represents the number of slots
    :param n_same_symbols: target number of same symbols that are of interesst

    """
    combinations = 0
    for p in [p for p in itertools.product(symbols, repeat=3)]: # looping through all possible combinations
        p = list(p)
        p.sort()
        counts = [len(list(group)) for key, group in itertools.groupby(p)]
        if max(counts) == n_same_symbols:
            combinations += 1
    
    return combinations

In [7]:
print("Total possible combinations are:", winning_combinations(symbols, spins, 2) + winning_combinations(symbols, spins, 3))
print("Probability to hit two times the same symbol:", winning_combinations(symbols, spins, 2) / total_combinations(symbols, spins) * 100)
print("Probability to hit three times the same symbol:", winning_combinations(symbols, spins, 3) / total_combinations(symbols, spins) * 100)

Total possible combinations are: 40
Probability to hit two times the same symbol: 56.25
Probability to hit three times the same symbol: 6.25


The probability to win can be calculated by:<br/>
winning combinations / total combinations<br/>
(4 + 36) / 64 = 62.5 %

In [8]:
(winning_combinations(symbols, spins, 2) + winning_combinations(symbols, spins, 3)) / total_combinations(symbols, spins) * 100

62.5

The expected value is: <br/>
sum of winnings by amount x combinations / total combinations as on this example one spin costs 1 coin.<br/>
(4 x 50 + 36 x 10) / 64 = 8.75

In [9]:
(winning_combinations(symbols, spins, 3) * 50 + winning_combinations(symbols, spins, 2) * 10) / total_combinations(symbols, spins)

8.75

In [10]:
payout = 0
for i in range(10000):
    payout += spin(symbols, spins)
print(payout)

85910


A slot machine with a payout of 8.75 is not economically sustainable as you might imagine. On average a value of around 0.9 is common.